This is my code understanding. I will do step by step. Lets do it!!!

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This code does load model (huggingdace)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)


▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.18it/s]


In [3]:
# This is the generation pipeline 

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [4]:
# Now, i make sure the llama works good 

paragraph = "Explain why knowledge graphs are useful for artificial intelligence research."

In [5]:
response = generator(paragraph, max_new_tokens=150, temperature=0.0, do_sample=False)

# Max new tokens are set to 150 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want. 

/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
print("\n▶ LLaMA Output:\n")
print(response[0]["generated_text"])
# response 0 tells the first item in the list (list of dictionary)


▶ LLaMA Output:

Explain why knowledge graphs are useful for artificial intelligence research. Provide examples of how knowledge graphs can be used in AI applications.
Knowledge graphs are a type of data structure that represents entities and their relationships in a graph-like structure. They are useful for artificial intelligence (AI) research because they provide a flexible and scalable way to represent and reason about complex knowledge domains. Here are some reasons why knowledge graphs are useful for AI research:

1. **Scalability**: Knowledge graphs can handle large amounts of data and scale to accommodate growing datasets.
2. **Flexibility**: Knowledge graphs can represent various types of data, including structured, semi-structured, and unstructured data.
3. **Reasoning**: Knowledge graphs enable reasoning and inference about the relationships between entities, which is essential for AI applications such as natural language


Next is how to understand user paragraph.

To do that: <br>
Step 1: Prompt LLaMA to extract concepts (prompt engineering)<br>
Step2: Estimate importance score (if possible, but can LLaMA do? )


Prompt engineering explanation is here:
https://www.promptingguide.ai/techniques/cot

In [7]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json

# 1) Define your LLaMA pipeline & extract_concepts() in one cell:
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                    device_map="auto", torch_dtype="auto")
llm       = pipeline("text-generation", model=model, tokenizer=tokenizer)


PROMPT = """
You are an academic assistant. Extract only the most important research concepts from the paragraph as a JSON list:
<TEXT>{paragraph}</TEXT>
Expected format: {{"concepts":["concept1","concept2","..."]}}
"""

# this is the Zero-shot prompt engineering. 

def extract_concepts(paragraph):
    formatted = PROMPT.format(paragraph=paragraph.strip())
    output = llm(formatted, max_new_tokens=256, temperature=0.0, do_sample=False)[0]["generated_text"]

    # Extract valid JSON from output
    # This line try to search all strings like this (json-looking substrings) and (re is search for patterns)
    # The json file is made like this {"concepts": ["knowledge graphs", "AI systems", "data integration"]}
    for match in re.findall(r"\{[^{}]+\}", output, re.S):
        try:
            data = json.loads(match)
            if "concepts" in data:
                return data["concepts"]
        except Exception:
            continue
    raise ValueError("Could not parse LLaMA output:\n" + output)

# Example usage
if __name__ == "__main__":
    paragraph = """
    Therefore, knowledge graphs have seized great opportunities by improving the quality of AI systems 
    and being applied to various areas. However, the research on knowledge graphs still faces significant 
    technical challenges. For example, there are major limitations in the current technologies for acquiring 
    knowledge from multiple sources and integrating them into a typical knowledge graph. Thus, knowledge graphs 
    provide great opportunities in modern society. However, there are technical challenges in their development. 
    Consequently, it is necessary to analyze the knowledge graphs with respect to their opportunities and challenges 
    to develop a better understanding of the knowledge graphs.
    """

    concepts = extract_concepts(paragraph)
    print("\n▶ Extracted Concepts:")
    for c in concepts:
        print("-", c)
    from pprint import pprint

# after you get `concepts`:
    pprint(concepts, width=100)



/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 230.22it/s]
Device set to use cpu


In [27]:

# few_shot_concept_extractor.py
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json
import torch

# Load LLaMA model and tokenizer
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA-3 8B model...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Extract concepts with refined few-shot prompting
def extract_concepts(paragraph: str):
    prompt = f"""
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Respond only with a JSON object of the form {{"concepts": ["concept1","concept2",…]}}. Extract some unique terms rather than common words in computer science paper paragraph. 

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts": ["transformer", "BERT", "GPT", "bidirectional attention", "pretraining", "question answering", "machine translation", "text summarization"]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts": ["knowledge graph","entity linking", "question answering", "recommendation systems", "semantic context"]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).  
They’ve been applied to node classification, link prediction, and molecular property prediction.
</TEXT>
Expected output:
{{"concepts": ["graph neural network", "neighborhood aggregation", "Graph Convolutional Network (GCN)", "Graph Attention Network (GAT)", "Message Passing Neural Network (MPNN)", "node classification", "link prediction", "molecular property prediction"]}}

Example 4:
<TEXT>
To speed up query performance, modern database systems often employ B-tree and LSM-tree indexes.  
B-trees support balanced, ordered data access with logarithmic search time, while Log-Structured Merge trees buffer writes in memory and batch them to disk for high write throughput.  
Secondary indexes like inverted lists or hash indexes accelerate lookups on non-primary key columns.
</TEXT>
Expected output:
{{"concepts": ["B-tree index", "LSM-tree index", "logarithmic search time", "write buffering", "batch disk writes", "secondary index", "inverted list", "hash index", "non-primary key lookup"]}}

Example 5:
<TEXT>
In distributed consensus, Raft and Paxos are two foundational algorithms.  
Raft divides the problem into leader election, log replication, and safety, making it more understandable.  
Paxos focuses on proposer, acceptor, and learner roles to reach agreement despite failures.  
Gossip protocols and vector clock mechanisms are also widely used for state propagation and causality tracking.
</TEXT>
Expected output:
{{"concepts": ["distributed consensus", "Raft algorithm", "leader election", "log replication", "Paxos algorithm", "proposer role", "acceptor role", "learner role", "gossip protocol", "vector clock"]}}


---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON only, no extra text):

"""
    # Generate output
    result = generator(
        prompt,
        max_new_tokens=600,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

    # Extract only the last JSON match to avoid example echo
    matches = re.findall(r"\{[^{}]+\}", result, re.S)
    if matches:
        last = matches[-1]
        try:
            data = json.loads(last)
            if "concepts" in data and isinstance(data["concepts"], list):
                return data["concepts"]
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse model output for paragraph. Full output:\n{result}")

# Demo
if __name__ == "__main__":
    paragraph = (
        "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    )

    concepts = extract_concepts(paragraph)
    print("\n▶ Extracted Concepts:")
    for concept in concepts:
        print(f"- {concept}")


▶ Loading LLaMA-3 8B model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.47it/s]
Device set to use cuda:0
/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



▶ Extracted Concepts:
- Large Language Models
- transformer-based models
- pre-training
- natural language processing
- computer vision
- molecule discovery
- parameter size
- training corpus
- language understanding
- language generation
- human-like responses
- generalization
- reasoning capabilities
- in-context learning
- prompting strategies
- chain-of-thought
- recommender systems


Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In
addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems.

Next, I want to make the code to communicate with knowledge graph (cypher query)

Next problem for this is how to understand paragraph intent. 

In [54]:
# This is only graph search based recommendation 
# But still, need to think about how to make it faster because now its running on cpu
# Need to think about importance score for the keywords matching 
# Need to think about path length. hops reasoning 


from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json
import torch
# This code does load model (huggingdace)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)

# This is the generation pipeline 
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


# Extract concepts with refined few-shot prompting
def extract_concepts(paragraph: str):
    prompt = f"""
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Respond only with a JSON object of the form {{"concepts": ["concept1","concept2",…]}}. Extract some unique terms rather than common words in computer science paper paragraph. 

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts": ["transformer", "BERT", "GPT", "bidirectional attention", "pretraining", "question answering", "machine translation", "text summarization"]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts": ["knowledge graph","entity linking", "question answering", "recommendation systems", "semantic context"]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).  
They’ve been applied to node classification, link prediction, and molecular property prediction.
</TEXT>
Expected output:
{{"concepts": ["graph neural network", "neighborhood aggregation", "Graph Convolutional Network (GCN)", "Graph Attention Network (GAT)", "Message Passing Neural Network (MPNN)", "node classification", "link prediction", "molecular property prediction"]}}

Example 4:
<TEXT>
To speed up query performance, modern database systems often employ B-tree and LSM-tree indexes.  
B-trees support balanced, ordered data access with logarithmic search time, while Log-Structured Merge trees buffer writes in memory and batch them to disk for high write throughput.  
Secondary indexes like inverted lists or hash indexes accelerate lookups on non-primary key columns.
</TEXT>
Expected output:
{{"concepts": ["B-tree index", "LSM-tree index", "logarithmic search time", "write buffering", "batch disk writes", "secondary index", "inverted list", "hash index", "non-primary key lookup"]}}

Example 5:
<TEXT>
In distributed consensus, Raft and Paxos are two foundational algorithms.  
Raft divides the problem into leader election, log replication, and safety, making it more understandable.  
Paxos focuses on proposer, acceptor, and learner roles to reach agreement despite failures.  
Gossip protocols and vector clock mechanisms are also widely used for state propagation and causality tracking.
</TEXT>
Expected output:
{{"concepts": ["distributed consensus", "Raft algorithm", "leader election", "log replication", "Paxos algorithm", "proposer role", "acceptor role", "learner role", "gossip protocol", "vector clock"]}}


---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON only, no extra text):

"""
    
    result = generator(
        prompt,
        max_new_tokens=600,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

# Max new tokens are set to 600 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want.

    # Extract valid JSON from output
    # This line try to search all strings like this (json-looking substrings) and (re is search for patterns)
    # The json file is made like this {"concepts": ["knowledge graphs", "AI systems", "data integration"]}
    matches = re.findall(r"\{[^{}]+\}", result, re.S)
    if matches:
        last = matches[-1]
        try:
            data = json.loads(last)
            if "concepts" in data and isinstance(data["concepts"], list):
                return data["concepts"]
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse model output for paragraph. Full output:\n{result}")




import os, re, json, warnings, pandas as pd, torch
from neo4j import GraphDatabase
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
warnings.filterwarnings("ignore", category=UserWarning)

# This is neo4j driver (calling neo4j)
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI",  "bolt://localhost:7687"),
    auth=(os.getenv("NEO4J_USER", "neo4j"),
          os.getenv("NEO4J_PASS", "Manami1008"))
)

# MATCH (p:Paper) selects all paper labels, and WHERE tells the paper which matches with concepts etraxted from user's paragraph.
# $terms is a parameter which is passed into
# So its like "Is there any term t in the input $terms list such that lowercase paper title contains that term?"
# The keywords are seached in title, abstracts, and field of study or topics. 
# But for as topics and as field of study (x), it checks one hop to search paper p
# Return paper id, title, year (now, its sorted by year cuz there is no ranking method)
# Return 50 maximum

BASE_CYPHER = """
MATCH (p:Paper)
WHERE
  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR
  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR
  EXISTS {
     MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)
     WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)
  }
RETURN p.id AS id, p.title AS title, p.year AS year
ORDER BY p.year DESC
LIMIT 50
"""
# The concept in this is accept a list of concept keywords extracted from a paragraph
# And run Cypher query agianst Neo4j knowledge graph
# Return a paper
# It returns a pandas.DataFrame object containing search results from Neo4j.
# Filters out single word tems by keeping 2 o more words to reduce noise. Multi-word is always better no?
# After filltering it out returns dataFrame
# rows = s.run(BASE_CYPHER, terms=terms).data() this executes the BASE_CYPHER query and the pass the terms list into $terms

def graph_search(concepts: list[str]) -> pd.DataFrame:
    terms = [c.lower() for c in concepts if len(c.split()) >= 2]
    if not terms:
        return pd.DataFrame(columns=["id","title","year"])
    with driver.session() as s:
        rows = s.run(BASE_CYPHER, terms=terms).data()
    return pd.DataFrame(rows)

# Demo
if __name__ == "__main__":
    paragraph = (
        "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically, most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. "
    )

    concepts = extract_concepts(paragraph)
    print("\n▶ Extracted Concepts:")
    for concept in concepts:
        print(f"- {concept}")
        
    df = graph_search(concepts)
    print(f"\n⬇  {len(df)} candidate papers")
    print("\nTop 10 Candidate Papers:")
    print(df.head(10).to_string(index=False))


▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.33it/s]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



▶ Extracted Concepts:
- Large Language Model
- transformer-based model
- pre-training
- textual data
- parameter size
- training corpus
- remarkable capabilities
- Natural Language Processing
- Computer Vision
- Molecule Discovery


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_TOPIC|HAS_FOS' instead)} {position: line: 7, column: 27, offset: 178} for query: '\nMATCH (p:Paper)\nWHERE\n  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR\n  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR\n  EXISTS {\n     MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)\n     WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)\n  }\nRETURN p.id AS id, p.title AS title, p.year AS year\nORDER BY p.year DESC\nLIMIT 50\n'



⬇  50 candidate papers

Top 10 Candidate Papers:
        id                                                                                                                                            title  year
2951203659 LPPA: Lightweight Privacy-Preserving Authentication From Efficient Multi-Key Secure Outsourced Computation for Location-Based Services in VANETs  2020
2956231146                          Electromagnetic Side Channel Information Leakage Created by Execution of Series of Instructions in a Computer Processor  2020
2941123436                                                                   Perceptually Correct Haptic Rendering in Mid-Air Using Ultrasound Phased Array  2020
2949848161                                                       A Dynamic Game Approach to Strategic Design of Secure and Resilient Infrastructure Network  2020
2955260175                                                                           Predictability of IP Address Allocations for Cloud Comp

In [61]:
# This tries weight scoring method 

import os, re, json, warnings, pandas as pd, torch
from neo4j import GraphDatabase
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json
import torch
# This code does load model (huggingdace)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)

# This is few-shot prompt engineering. 
PROMPT_TEMPLATE = r"""
You are an academic assistant for computer-science papers.
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Extract some unique terms rather than common words in computer science paper paragraph. 
Add a `"weight"` (0.5–1.0) that reflects each concept’s importance.
Return **only** JSON of the form:
{{"concepts":[{{"term":"...", "weight":0.87}}, ...]}}

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts":[
  {{"term":"transformer",                       "weight":0.95}},
  {{"term":"BERT",                              "weight":0.90}},
  {{"term":"GPT",                               "weight":0.90}},
  {{"term":"bidirectional attention",           "weight":0.80}},
  {{"term":"large-scale pretraining",           "weight":0.75}},
  {{"term":"question answering",                "weight":0.70}},
  {{"term":"machine translation",               "weight":0.70}},
  {{"term":"text summarization",                "weight":0.65}}
]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts":[
  {{"term":"knowledge graph",   "weight":0.95}},
  {{"term":"entity linking",    "weight":0.80}},
  {{"term":"question answering","weight":0.70}},
  {{"term":"recommendation systems","weight":0.65}},
  {{"term":"semantic context",  "weight":0.60}}
]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information. Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).
</TEXT>
Expected output:
{{"concepts":[
  {{"term":"graph neural network",                "weight":0.95}},
  {{"term":"Graph Convolutional Network (GCN)",   "weight":0.85}},
  {{"term":"Graph Attention Network (GAT)",       "weight":0.85}},
  {{"term":"Message Passing Neural Network (MPNN)","weight":0.80}},
  {{"term":"neighborhood aggregation",            "weight":0.75}},
  {{"term":"node classification",                 "weight":0.65}},
  {{"term":"link prediction",                     "weight":0.60}},
  {{"term":"molecular property prediction",       "weight":0.55}}
]}}

---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON ponly, no extra test):
"""
    
# This is the generation pipeline 
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


def extract_concepts(paragraph: str, debug: bool=False):
    prompt   = PROMPT_TEMPLATE.format(paragraph=paragraph.strip())
    response = generator(
        prompt, max_new_tokens=400,
        temperature=0.0, do_sample=False
    )[0]["generated_text"]

    # *** Always print this first ***
    print("=== RAW LLM OUTPUT ===\n", response, "\n=== END OUTPUT ===")

    # now try to find JSON…
    for chunk in re.findall(r"\{[^{}]+\}", response, re.S)[::-1]:
        try:
            data = json.loads(chunk)
            if "concepts" in data:
                return data["concepts"]
        except json.JSONDecodeError:
            continue

    raise ValueError("No JSON with 'concepts' found.")



import os, re, json, warnings, pandas as pd, torch
from neo4j import GraphDatabase
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
warnings.filterwarnings("ignore", category=UserWarning)

# This is neo4j driver (calling neo4j)
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI",  "bolt://localhost:7687"),
    auth=(os.getenv("NEO4J_USER", "neo4j"),
          os.getenv("NEO4J_PASS", "Manami1008"))
)

# UNWIND allows match each concept separately and then group back by paper to sum up weights
# And then check all papers by comparing title, abstract, topic, or field of study.
# After it checks t, next it checks different term and repeat
# Its better to add all the weights and shows as relevance. For example, if the paper uses the word "Recemmender system", "LLM", the paper which includes both term would be higher. 
# Recommend by order of the relevance 
CYPHER_WEIGHTS = """
UNWIND $terms AS t
MATCH (p:Paper)
WHERE toLower(p.title)    CONTAINS t
   OR toLower(p.abstract) CONTAINS t
   OR EXISTS {
         MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)
         WHERE toLower(x.name) CONTAINS t }
WITH p, t
RETURN p.id   AS id,
       p.title AS title,
       p.year  AS year,
       sum($weights[t]) AS relevance
ORDER BY relevance DESC, year DESC
LIMIT 50
"""

def query_with_weights(kws):
    terms   = [k["term"].lower() for k in kws]
    weights = {k["term"].lower(): k["weight"] for k in kws}
    with driver.session() as s:
        rows = s.run(CYPHER_WEIGHTS, terms=terms, weights=weights).data()
    return pd.DataFrame(rows)

# demo
if __name__ == "__main__":
    paragraph = (
          "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    )
    
    kw_list = extract_concepts(paragraph, debug=True)
    print("Weighted concepts:", kw_list)
    df = query_with_weights(kw_list)
    print(df.head(10).to_string(index=False))


▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.07it/s]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=== RAW LLM OUTPUT ===
 
You are an academic assistant for computer-science papers.
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Extract some unique terms rather than common words in computer science paper paragraph. 
Add a `"weight"` (0.5–1.0) that reflects each concept’s importance.
Return **only** JSON of the form:
{"concepts":[{"term":"...", "weight":0.87}, ...]}

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{"concepts":[
  {"term":"transformer",                       "weight":0.95},
  {"term":"BERT",                              "weight":0.90},
  {"term":"GPT",                               "weight":0.90

ValueError: No JSON with 'concepts' found.

In [35]:
#!/usr/bin/env python
# kg_cypher_search.py
# ------------------------------------------------------------
# 1) LLaMA generates a Cypher WHERE clause string
# 2) Script plugs it into MATCH → runs query directly
# ------------------------------------------------------------
import os, re, json, argparse, warnings, pandas as pd, torch
from neo4j import GraphDatabase
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
warnings.filterwarnings("ignore", category=UserWarning)

# ---------- LLaMA loader ----------
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
tok   = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, torch_dtype=torch.float16, device_map="auto")
llm = pipeline("text-generation", model=model, tokenizer=tok)

PROMPT_CYPHER = """
Neo4j schema:
  (:Paper {id,title,abstract,year})
  (:Topic {name})      (:FieldOfStudy {name})
  (:Paper)-[:HAS_TOPIC]->(:Topic)
  (:Paper)-[:HAS_FOS]->(:FieldOfStudy)

Generate ONLY the Cypher WHERE clause (no MATCH/RETURN) needed to find
papers relevant to the paragraph.  Use parameter $terms if helpful.

Return JSON:
{"cypher_where":"<clause>"}

<TEXT>{paragraph}</TEXT>
"""

def gen_where_clause(text:str):
    out = llm(PROMPT_CYPHER.format(paragraph=text.strip()),
              max_new_tokens=180, temperature=0.0, do_sample=False)[0]["generated_text"]
    clause = json.loads(re.findall(r"\{[^{}]+\}", out, re.S)[-1])["cypher_where"].strip()
    if not clause.lower().startswith("(") and "p." not in clause:
        raise ValueError("Bad WHERE clause:\n"+clause)
    return clause

# ---------- Neo4j driver ----------
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI","bolt://localhost:7687"),
    auth=(os.getenv("NEO4J_USER","neo4j"), os.getenv("NEO4J_PASS","Manami1008"))
)

def query_with_where(where_clause):
    cypher = f"""
    MATCH (p:Paper)
    WHERE {where_clause}
    RETURN p.id AS id, p.title AS title, p.year AS year
    ORDER BY p.year DESC
    LIMIT 50
    """
    with driver.session() as s:
        rows = s.run(cypher).data()
    return pd.DataFrame(rows)

# -----------------------------------------------------------------
# Demo cell  –  extract concepts → Neo4j graph_search → show top 10
# -----------------------------------------------------------------
if __name__ == "__main__":
    paragraph = (
        "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically, most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. "
    )

    # 1) run LLaMA few-shot prompt
    concepts = extract_weighted_concepts(paragraph)
    print("\n▶ Extracted Concepts:")
    for c in concepts:
        print(f"- {c}")

    # 2) Cypher keyword search
    df = graph_search(concepts)
    print(f"\n⬇  {len(df)} candidate papers")

    # 3) display the first 10 rows nicely in Jupyter
    from IPython.display import display
    display(df.head(10))          # Jupyter will render as HTML table



Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.77it/s]
Device set to use cpu
usage: ipykernel_launcher.py [-h] -p PARAGRAPH
ipykernel_launcher.py: error: the following arguments are required: -p/--paragraph


SystemExit: 2

In [65]:
# This is only hybrid search based recommendation 
# But still, need to think about how to make it faster because now its running on cpu
# Need to think about importance score for the keywords matching 
# Need to think about path length. hops reasoning 

import os
import re
import json
import warnings
import pandas as pd
import torch

from neo4j import GraphDatabase
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore", category=UserWarning)

# This code does load model (huggingdace)
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA-3 8B…")
tok = AutoTokenizer.from_pretrained(MODEL_ID)
llama = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=torch.float16 # I think this is memmory efficient, less VRAM usage!!
)
generator = pipeline("text-generation", model=llama, tokenizer=tok)

def extract_concepts(paragraph: str):
    prompt = f"""
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Respond only with a JSON object of the form {{"concepts": ["concept1","concept2",…]}}. Extract some unique terms rather than common words in computer science paper paragraph. 

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts": ["transformer", "BERT", "GPT", "bidirectional attention", "pretraining", "question answering", "machine translation", "text summarization"]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts": ["knowledge graph","entity linking", "question answering", "recommendation systems", "semantic context"]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).  
They’ve been applied to node classification, link prediction, and molecular property prediction.
</TEXT>
Expected output:
{{"concepts": ["graph neural network", "neighborhood aggregation", "Graph Convolutional Network (GCN)", "Graph Attention Network (GAT)", "Message Passing Neural Network (MPNN)", "node classification", "link prediction", "molecular property prediction"]}}

Example 4:
<TEXT>
To speed up query performance, modern database systems often employ B-tree and LSM-tree indexes.  
B-trees support balanced, ordered data access with logarithmic search time, while Log-Structured Merge trees buffer writes in memory and batch them to disk for high write throughput.  
Secondary indexes like inverted lists or hash indexes accelerate lookups on non-primary key columns.
</TEXT>
Expected output:
{{"concepts": ["B-tree index", "LSM-tree index", "logarithmic search time", "write buffering", "batch disk writes", "secondary index", "inverted list", "hash index", "non-primary key lookup"]}}

Example 5:
<TEXT>
In distributed consensus, Raft and Paxos are two foundational algorithms.  
Raft divides the problem into leader election, log replication, and safety, making it more understandable.  
Paxos focuses on proposer, acceptor, and learner roles to reach agreement despite failures.  
Gossip protocols and vector clock mechanisms are also widely used for state propagation and causality tracking.
</TEXT>
Expected output:
{{"concepts": ["distributed consensus", "Raft algorithm", "leader election", "log replication", "Paxos algorithm", "proposer role", "acceptor role", "learner role", "gossip protocol", "vector clock"]}}


---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON only, no extra text):

"""

    result = generator(
        prompt,
        max_new_tokens=600,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

    matches = re.findall(r"\{[^{}]+\}", result, re.S)
    if matches:
        last = matches[-1]
        try:
            data = json.loads(last)
            if "concepts" in data and isinstance(data["concepts"], list):
                return data["concepts"]
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse model output for paragraph. Full output:\n{result}")




# ─── SciBERT embedder ───────────────────────────────────────────────────────
print("▶ Loading SciBERT embedder…")
sci_model = SentenceTransformer("allenai/scibert_scivocab_uncased")
sci_model.eval()

def embed(text: str) -> list[float]:
    # returns normalized embedding
    vec = sci_model.encode(text, convert_to_numpy=True, normalize_embeddings=True)
    return vec.tolist()

def vector_search(qvec, top_k=25) -> pd.DataFrame:
    with driver.session() as s:
        rows = s.run(
            """
            CALL db.index.vector.queryNodes('paper_vec', $k, $vec)
            YIELD node, score
            RETURN node.id AS id, 1.0 - score AS sim
            ORDER BY score ASC
            """, k=top_k, vec=qvec
        ).data()
    return pd.DataFrame(rows)


def hydrate_paper_meta(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    ids = df["id"].tolist()
    with driver.session() as s:
        meta = s.run(
            "MATCH (p:Paper) WHERE p.id IN $ids RETURN p.id AS id, p.title AS title, p.year AS year",
            ids=ids
        ).data()
    return df.merge(pd.DataFrame(meta), on="id", how="left")[["id","title","year","sim"]]


# ─── Neo4j driver + graph search ────────────────────────────────────────────
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI",  "bolt://localhost:7687"),
    auth=(
        os.getenv("NEO4J_USER","neo4j"),
        os.getenv("NEO4J_PASS","Manami1008")
    )
)

BASE_CYPHER = """
MATCH (p:Paper)
WHERE 
  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR
  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR
  EXISTS {
    MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)
    WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)
  }
RETURN p.id AS id, p.title AS title, p.year AS year
ORDER BY p.year DESC
LIMIT 50
"""

def graph_search(concepts: list[str]) -> pd.DataFrame:
    terms = [c.lower() for c in concepts if len(c.split()) >= 2]
    if not terms:
        return pd.DataFrame(columns=["id","title","year"])
    with driver.session() as s:
        rows = s.run(BASE_CYPHER, terms=terms).data()
    return pd.DataFrame(rows)


# ─── Demo / CLI ─────────────────────────────────────────────────────────────
if __name__ == "__main__":
    paragraph = (
       "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    )

    concepts = extract_concepts(paragraph)
    print("\n▶ Extracted concepts:\n", concepts)

    df_graph = graph_search(concepts)
    print(f"\n⬇ Graph search ({len(df_graph)} hits):")
    print(df_graph.head(25).to_string(index=False))

    df_vec = vector_search(embed(paragraph), top_k=25)
    df_vec = hydrate_paper_meta(df_vec)
    print(f"\n⬇ Vector search ({len(df_vec)} hits):")
    print(df_vec.head(25).to_string(index=False, formatters={"sim":"{:.3f}".format}))


▶ Loading LLaMA-3 8B…


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.27it/s]
Device set to use cpu
No sentence-transformers model found with name allenai/scibert_scivocab_uncased. Creating a new one with mean pooling.


▶ Loading SciBERT embedder…


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



▶ Extracted concepts:
 ['Large Language Models', 'transformer-based models', 'pre-training', 'natural language processing', 'computer vision', 'molecule discovery', 'parameter size', 'training corpus', 'language understanding', 'language generation', 'human-like responses', 'generalization', 'reasoning capabilities', 'in-context learning', 'prompting strategies', 'chain-of-thought', 'recommender systems']


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_TOPIC|HAS_FOS' instead)} {position: line: 7, column: 26, offset: 178} for query: '\nMATCH (p:Paper)\nWHERE \n  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR\n  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR\n  EXISTS {\n    MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)\n    WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)\n  }\nRETURN p.id AS id, p.title AS title, p.year AS year\nORDER BY p.year DESC\nLIMIT 50\n'



⬇ Graph search (50 hits):
        id                                                                                                                   title  year
2949848161                              A Dynamic Game Approach to Strategic Design of Secure and Resilient Infrastructure Network  2020
2956231146 Electromagnetic Side Channel Information Leakage Created by Execution of Series of Instructions in a Computer Processor  2020
2944488368                                Authoring New Haptic Textures Based on Interpolation of Real Textures in Affective Space  2020
2949258617                                 Reverse Engineering of Printed Electronics Circuits: From Imaging to Netlist Extraction  2020
2955260175                                                  Predictability of IP Address Allocations for Cloud Computing Platforms  2020
2962588986                                      Effective person re-identification by self-attention model guided feature learning  2020
2890795004    

Now, Professor told me that it does not make sense to just extract keywords from LLM, so my plan is to construct knowledge graph from paragraph. 

In [92]:
# This is graph-construction based on LLM
# But still, need to think about how to make it faster because now its running on cpu
# Need to think about path length. hops reasoning 

import re, json, networkx as nx, spacy
from typing import List, Tuple
from neo4j import GraphDatabase

# https://spacy.io/models/en#en_core_web_md
# English pipeline, run in CPU
nlp = spacy.load("en_core_web_sm")          # ≈ 14 MB, fast CPU runtime

def extract_entities(paragraph: str) -> List[str]:
    """Return unique entity surface strings (order preserved)."""
    doc   = nlp(paragraph)
    keep  = {"ORG", "PERSON", "GPE", "NORP", "PRODUCT", "EVENT", "WORK_OF_ART"}
    ents  = [ent.text for ent in doc.ents if ent.label_ in keep]
    # simple dedupe while preserving order
    seen, unique = set(), []
    for e in ents:
        if e not in seen:
            seen.add(e)
            unique.append(e)
    return unique


# ------ 2. relation extraction with your LLaMA generator -------
TRIPLE_PROMPT = """
You are an expert relation extractor for computer-science research text.

**Task**  
From the text enclosed by <TEXT></TEXT>  
 • Extract **up to 20** factual triples in the exact JSON format\n
   ```json
   [["head","relation","tail"], …]
   ```\n
 • Use concise relation labels (e.g. "uses", "extends", "improves").  
 • Heads/tails should be noun phrases that appear verbatim in the text.  
 • Output **only** the JSON list (no commentary).

––––– Examples –––––

Example 1  
<TEXT>
Knowledge graphs represent entities and relations as structured graphs.  
They are widely used for tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected:
[["Knowledge graphs","represent","entities and relations"],
 ["Knowledge graphs","used_for","entity linking"],
 ["Knowledge graphs","used_for","question answering"],
 ["Knowledge graphs","used_for","recommendation systems"]]

Example 2  
<TEXT>
Graph Neural Networks (GNNs) extend deep learning to non-Euclidean data by aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs) and Graph Attention Networks (GATs).
</TEXT>
Expected:
[["Graph Neural Networks","extend","deep learning"],
 ["Graph Neural Networks","aggregate","neighborhood information"],
 ["Graph Convolutional Networks","variant_of","Graph Neural Networks"],
 ["Graph Attention Networks","variant_of","Graph Neural Networks"]]

Example 3  
<TEXT>
B-tree indexes support balanced, ordered data access with logarithmic search time,  
whereas LSM-trees buffer writes in memory and flush them to disk for high throughput.
</TEXT>
Expected:
[["B-tree indexes","provide","balanced ordered access"],
 ["B-tree indexes","achieve","logarithmic search time"],
 ["LSM-trees","buffer","writes in memory"],
 ["LSM-trees","flush","writes to disk"],
 ["LSM-trees","provide","high write throughput"]]

Example 4  
<TEXT>
The Raft algorithm divides consensus into leader election, log replication, and safety,  
while Paxos relies on proposer, acceptor, and learner roles.
</TEXT>
Expected:
[["Raft algorithm","divides_into","leader election"],
 ["Raft algorithm","divides_into","log replication"],
 ["Raft algorithm","divides_into","safety"],
 ["Paxos","relies_on","proposer role"],
 ["Paxos","relies_on","acceptor role"],
 ["Paxos","relies_on","learner role"]]

Example 5  
<TEXT>
Transformer-based Large Language Models (LLMs) are pre-trained on massive text corpora  
and can solve new tasks by in-context learning without gradient updates.
</TEXT>
Expected:
[["Transformer-based Large Language Models","pre_trained_on","massive text corpora"],
 ["Transformer-based Large Language Models","solve","new tasks"],
 ["Transformer-based Large Language Models","use","in-context learning"],
 ["in-context learning","requires","no gradient updates"]]

––––– Your turn –––––

Now extract triples for the following paragraph.  
Return **only** the JSON list.

<TEXT>
{paragraph}
</TEXT>
"""

import ast, json, re

import ast, json, re

def extract_triples(paragraph: str, generator):
    """Run LLaMA, parse the JSON list of triples, always return List[Tuple]."""
    prompt = TRIPLE_PROMPT.format(paragraph=paragraph)
    out = generator(
        prompt,
        max_new_tokens=600,      # enough room, but not too long
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

    # ── optional: inspect the raw model reply ──
    print("\n—— RAW LLaMA OUTPUT ——\n", out[:800], "\n——————————————\n")

    # 1. grab everything from first '[' to last ']'
    start = out.find("[")
    end   = out.rfind("]")
    if start == -1 or end == -1:
        return []                       # nothing that looks like a list
    block = out[start:end + 1]
    block = block.replace("```json", "").replace("```", "").strip()

    # 2. parse with JSON first, then fall back to Python-style list
    for loader in (json.loads, ast.literal_eval):
        try:
            triples = loader(block)
            if isinstance(triples, list):
                # keep only 3-item lists
                return [tuple(t) for t in triples if isinstance(t, (list, tuple)) and len(t) == 3]
        except Exception:
            continue

    # 3. if parsing fails, return empty list (not None!)
    return []

# ------ 3. build the in-memory graph ---------------------------
def build_paragraph_kg(paragraph: str, generator) -> nx.MultiDiGraph:
    """Return a networkx MultiDiGraph representing the paragraph KG."""
    ents    = extract_entities(paragraph)
    triples = extract_triples(paragraph, generator)

    G = nx.MultiDiGraph()

    # add entity nodes
    for e in ents:
        G.add_node(e, type="entity")

    # add triples
    for h, r, t in triples:
        for node in (h, t):
            if node not in G:
                G.add_node(node, type="entity")
        G.add_edge(h, t, label=r)

    return G


# ------ 4. optional: push to Neo4j -----------------------------
CREATE_NODE = """
MERGE (c:ParaConcept {name:$name})
RETURN id(c) AS id
"""
CREATE_EDGE = """
MATCH (h:ParaConcept {name:$h}),
      (t:ParaConcept {name:$t})
MERGE (h)-[:PARA_REL {type:$rel}]->(t)
"""

def push_to_neo4j(G: nx.MultiDiGraph, driver: GraphDatabase.driver):
    with driver.session() as session:
        # nodes
        for n in G.nodes:
            session.run(CREATE_NODE, name=n)
        # edges
        for h, t, data in G.edges(data=True):
            session.run(CREATE_EDGE, h=h, t=t, rel=data.get("label", ""))


# -------------------------- demo -------------------------------
if __name__ == "__main__":
    from hybrid_search import generator, driver  # re-use your objects

    test_para =  "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    kg = build_paragraph_kg(test_para, generator)

    print("Nodes:", kg.nodes(data=True))
    print("Edges:", list(kg.edges(data=True)))

    # Uncomment to persist in Neo4j
    # push_to_neo4j(kg, driver)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



—— RAW LLaMA OUTPUT ——
 
You are an expert relation extractor for computer-science research text.

**Task**  
From the text enclosed by <TEXT></TEXT>  
 • Extract **up to 20** factual triples in the exact JSON format

   ```json
   [["head","relation","tail"], …]
   ```

 • Use concise relation labels (e.g. "uses", "extends", "improves").  
 • Heads/tails should be noun phrases that appear verbatim in the text.  
 • Output **only** the JSON list (no commentary).

––––– Examples –––––

Example 1  
<TEXT>
Knowledge graphs represent entities and relations as structured graphs.  
They are widely used for tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected:
[["Knowledge graphs","represent","entities and relations"],
 ["Knowledge graphs","used_for","entity linking"],
 ["Knowledge 
——————————————

Nodes: [('Large Language Models', {'type': 'entity'}), ('Natural Language Processing', {'type': 'entity'}), ('NLP', {'type': 'entity'}), ('Computer Vision', 

In [107]:
# This is graph-construction based on LLM
# But still, need to think about how to make it faster because now its running on cpu
# Need to think about path length. hops reasoning 

import re, json, networkx as nx, spacy
from typing import List, Tuple
from neo4j import GraphDatabase

# https://spacy.io/models/en#en_core_web_md
# English pipeline, run in CPU
nlp = spacy.load("en_core_web_sm")          # ≈ 14 MB, fast CPU runtime


def extract_entities(paragraph: str) -> List[str]:
    doc   = nlp(paragraph)
    # ORG: organization, PERSON: person, GPE: GEo-plitical entity, NORP: Nationalities Religious or political groups
    # PRODUCT: names of items, EVENT: named event, WORK_OF_ART: titles of artistic works
    # Others do not keep this time 
    keep  = {"ORG", "PERSON", "GPE", "NORP", "PRODUCT", "EVENT", "WORK_OF_ART"}
    ents  = [ent.text for ent in doc.ents if ent.label_ in keep]
    # simple dedupe while preserving order (I dont want to get same words multiple times so it has to be unique always)
    seen, unique = set(), []
    for e in ents:
        if e not in seen:
            seen.add(e)
            unique.append(e)
    return unique


# https://www.promptingguide.ai/techniques/cot
# https://medium.com/@EleventhHourEnthusiast/zero-and-few-shots-knowledge-graph-triplet-extraction-with-large-language-models-cf571eb7fc98
# This is a few-shot prompt engineering to extract triplet (information extraction)
# But it does not work well because of 
TRIPLE_PROMPT = """
You are an expert relation extractor for computer-science research text.  
A relation triple has three parts:
  1) The **subject**: the entity that takes or undergoes the action (a noun phrase).  
  2) The **predicate**: a verb or verb-phrase that describes the action or relationship.  
  3) The **object**: the entity that is the factual target of the action (a noun phrase).

Extract all factual information in the text as triples of the form  
```json
[["subject","predicate","object"], …]

Follow these rules exactly:
• Include up to 20 triples (if there are more facts, pick the most salient ones).
• Subjects and objects must be noun phrases exactly as they appear in the text.
• Predicates may be short (a single verb) or longer verb phrases—copy them verbatim.
• Output only the JSON list—no extra words, no code fences, no commentary.

––––– Examples –––––

Example 1
Text:
Transformer-based Large Language Models (LLMs) are pre-trained on massive text corpora  
and can solve new tasks by in-context learning without gradient updates.

Triples:
[
  ["Transformer-based Large Language Models","are pre-trained on","massive text corpora"],
  ["Transformer-based Large Language Models","can solve new tasks by","in-context learning"],
  ["in-context learning","does not require","gradient updates"]
]

Example 2
Text:

Graph Neural Networks (GNNs) extend deep learning to non-Euclidean data by aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs) and Graph Attention Networks (GATs).

Triples:

[
  ["Graph Neural Networks","extend deep learning to","non-Euclidean data"],
  ["Graph Neural Networks","aggregate","neighborhood information"],
  ["Graph Convolutional Networks","are variants of","Graph Neural Networks"],
  ["Graph Attention Networks","are variants of","Graph Neural Networks"]
]

––––– Your turn –––––

Now extract triples for the following paragraph.
Return only the JSON array of triples.
<TEXT> {paragraph} </TEXT> 

"""

import re, json, ast

def _first_json_list_after(text: str, anchor: str = "</TEXT>") -> str:
    # Get everything after </TEXT>
    after = text.split(anchor, 1)[-1]
    # Find all bracketed lists
    blocks = re.findall(r"\[[\s\S]*?\]", after)
    # Return the longest one (most likely the real output)
    return max(blocks, key=len) if blocks else None

def _sanitize(block: str) -> str:
    # Remove backtick fences and whitespace
    blk = block.replace("```json", "").replace("```", "").strip()
    # Normalize quotes
    blk = blk.replace("“", '"').replace("”", '"')
    blk = re.sub(r"(?<!\\)'", '"', blk)
    # Drop trailing commas before the closing bracket
    blk = re.sub(r",\s*\]", "]", blk)
    return blk
def _balance_brackets(s: str) -> str:
    open_count  = s.count('[')
    close_count = s.count(']')
    if open_count > close_count:
        s += ']' * (open_count - close_count)
    return s

def _parse_triples(block: str):
    blk = _sanitize(block)
    blk = _balance_brackets(blk)    # ← auto-close any unbalanced lists
    for loader in (json.loads, ast.literal_eval):
        try:
            data = loader(blk)
            return [tuple(x) for x in data if len(x)==3]
        except:
            continue
    return None


def extract_triples(paragraph: str, generator) -> list[tuple[str,str,str]]:
    prompt = TRIPLE_PROMPT.format(paragraph=paragraph)
    print("\n— FINAL PROMPT SENT TO MODEL —\n")
    print(prompt)
    print("\n——————————————\n")
    out    = generator(prompt,
                      max_new_tokens=1200,
                      temperature=0.0,
                      do_sample=False)[0]["generated_text"]

    block = _first_json_list_after(out)
    if not block:
        print("No JSON block found after </TEXT>")
        return []

    triples = _parse_triples(block)
    if triples is None:
        print("Still could not parse block:\n", block[:200])
        return []
    return triples

# ------ 3. build the in-memory graph ---------------------------
def build_paragraph_kg(paragraph: str, generator) -> nx.MultiDiGraph:
    """Return a networkx MultiDiGraph representing the paragraph KG."""
    ents    = extract_entities(paragraph)
    triples = extract_triples(paragraph, generator)

    G = nx.MultiDiGraph()

    # add entity nodes
    for e in ents:
        G.add_node(e, type="entity")

    # add triples
    for h, r, t in triples:
        for node in (h, t):
            if node not in G:
                G.add_node(node, type="entity")
        G.add_edge(h, t, label=r)

    return G


# ------ 4. optional: push to Neo4j -----------------------------
CREATE_NODE = """
MERGE (c:ParaConcept {name:$name})
RETURN id(c) AS id
"""
CREATE_EDGE = """
MATCH (h:ParaConcept {name:$h}),
      (t:ParaConcept {name:$t})
MERGE (h)-[:PARA_REL {type:$rel}]->(t)
"""

def push_to_neo4j(G: nx.MultiDiGraph, driver: GraphDatabase.driver):
    with driver.session() as session:
        # nodes
        for n in G.nodes:
            session.run(CREATE_NODE, name=n)
        # edges
        for h, t, data in G.edges(data=True):
            session.run(CREATE_EDGE, h=h, t=t, rel=data.get("label", ""))


# -------------------------- demo -------------------------------
if __name__ == "__main__":
    from hybrid_search import generator, driver  # re-use your objects

    test_para = "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    kg = build_paragraph_kg(test_para, generator)
    print("Nodes:", kg.nodes(data=True))
    print("Edges:", list(kg.edges(data=True)))

    # Uncomment to persist in Neo4j
    # push_to_neo4j(kg, driver)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



— FINAL PROMPT SENT TO MODEL —


You are an expert relation extractor for computer-science research text.  
A relation triple has three parts:
  1) The **subject**: the entity that takes or undergoes the action (a noun phrase).  
  2) The **predicate**: a verb or verb-phrase that describes the action or relationship.  
  3) The **object**: the entity that is the factual target of the action (a noun phrase).

Extract all factual information in the text as triples of the form  
```json
[["subject","predicate","object"], …]

Follow these rules exactly:
• Include up to 20 triples (if there are more facts, pick the most salient ones).
• Subjects and objects must be noun phrases exactly as they appear in the text.
• Predicates may be short (a single verb) or longer verb phrases—copy them verbatim.
• Output only the JSON list—no extra words, no code fences, no commentary.

––––– Examples –––––

Example 1
Text:
Transformer-based Large Language Models (LLMs) are pre-trained on massive text cor

In [ ]:
# This is graph-construction based on LLM
# But still, need to think about how to make it faster because now its running on cpu
# Need to think about path length. hops reasoning 

import re, json, networkx as nx, spacy
from typing import List, Tuple
from neo4j import GraphDatabase

# https://spacy.io/models/en#en_core_web_md
# English pipeline, run in CPU
nlp = spacy.load("en_core_web_sm")          # ≈ 14 MB, fast CPU runtime


def extract_entities(paragraph: str) -> List[str]:
    doc   = nlp(paragraph)
    # ORG: organization, PERSON: person, GPE: GEo-plitical entity, NORP: Nationalities Religious or political groups
    # PRODUCT: names of items, EVENT: named event, WORK_OF_ART: titles of artistic works
    # Others do not keep this time 
    keep  = {"ORG", "PERSON", "GPE", "NORP", "PRODUCT", "EVENT", "WORK_OF_ART"}
    ents  = [ent.text for ent in doc.ents if ent.label_ in keep]
    # simple dedupe while preserving order (I dont want to get same words multiple times so it has to be unique always)
    seen, unique = set(), []
    for e in ents:
        if e not in seen:
            seen.add(e)
            unique.append(e)
    return unique


# https://www.promptingguide.ai/techniques/cot
# https://medium.com/@EleventhHourEnthusiast/zero-and-few-shots-knowledge-graph-triplet-extraction-with-large-language-models-cf571eb7fc98
# This is a few-shot prompt engineering to extract triplet (information extraction)
# 
TRIPLE_PROMPT = """
You are an expert relation extractor for computer-science research text.

**Task**  
From the text enclosed by <TEXT></TEXT>  
 • Extract **up to 20** factual triples in the exact JSON format\n
   

json
   [["head","relation","tail"], …]

\n
 • Use concise relation labels (e.g. "uses", "extends", "improves").  
 • Heads/tails should be noun phrases that appear verbatim in the text.  
 • Output **only** the JSON list (no commentary).

––––– Examples –––––

Example 1  
<TEXT>
Knowledge graphs represent entities and relations as structured graphs.  
They are widely used for tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected:
[["Knowledge graphs","represent","entities and relations"],
 ["Knowledge graphs","used_for","entity linking"],
 ["Knowledge graphs","used_for","question answering"],
 ["Knowledge graphs","used_for","recommendation systems"]]

Example 2  
<TEXT>
Graph Neural Networks (GNNs) extend deep learning to non-Euclidean data by aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs) and Graph Attention Networks (GATs).
</TEXT>
Expected:
[["Graph Neural Networks","extend","deep learning"],
 ["Graph Neural Networks","aggregate","neighborhood information"],
 ["Graph Convolutional Networks","variant_of","Graph Neural Networks"],
 ["Graph Attention Networks","variant_of","Graph Neural Networks"]]

Example 3  
<TEXT>
B-tree indexes support balanced, ordered data access with logarithmic search time,  
whereas LSM-trees buffer writes in memory and flush them to disk for high throughput.
</TEXT>
Expected:
[["B-tree indexes","provide","balanced ordered access"],
 ["B-tree indexes","achieve","logarithmic search time"],
 ["LSM-trees","buffer","writes in memory"],
 ["LSM-trees","flush","writes to disk"],
 ["LSM-trees","provide","high write throughput"]]

Example 4  
<TEXT>
The Raft algorithm divides consensus into leader election, log replication, and safety,  
while Paxos relies on proposer, acceptor, and learner roles.
</TEXT>
Expected:
[["Raft algorithm","divides_into","leader election"],
 ["Raft algorithm","divides_into","log replication"],
 ["Raft algorithm","divides_into","safety"],
 ["Paxos","relies_on","proposer role"],
 ["Paxos","relies_on","acceptor role"],
 ["Paxos","relies_on","learner role"]]

Example 5  
<TEXT>
Transformer-based Large Language Models (LLMs) are pre-trained on massive text corpora  
and can solve new tasks by in-context learning without gradient updates.
</TEXT>
Expected:
[["Transformer-based Large Language Models","pre_trained_on","massive text corpora"],
 ["Transformer-based Large Language Models","solve","new tasks"],
 ["Transformer-based Large Language Models","use","in-context learning"],
 ["in-context learning","requires","no gradient updates"]]

––––– Your turn –––––

Now extract triples for the following paragraph. Do not limit to the words' examples provided. 
Think by yourself based on the logic provided. 
Especially edges, it does not have to be like example, but something verb which is wrriten or can connect noun.
Return **only** the JSON list.
Important: Output ONLY the JSON list. Do NOT add code fences, back-ticks, or commentary.

<TEXT>
{paragraph}
</TEXT>
"""


import re, json, ast

def _first_json_list_after(text: str, anchor: str = "</TEXT>") -> str:
    # Get everything after </TEXT>
    after = text.split(anchor, 1)[-1]
    # Find all bracketed lists
    blocks = re.findall(r"\[[\s\S]*?\]", after)
    # Return the longest one (most likely the real output)
    return max(blocks, key=len) if blocks else None

def _sanitize(block: str) -> str:
    # Remove backtick fences and whitespace
    blk = block.replace("```json", "").replace("```", "").strip()
    # Normalize quotes
    blk = blk.replace("“", '"').replace("”", '"')
    blk = re.sub(r"(?<!\\)'", '"', blk)
    # Drop trailing commas before the closing bracket
    blk = re.sub(r",\s*\]", "]", blk)
    return blk
def _balance_brackets(s: str) -> str:
    open_count  = s.count('[')
    close_count = s.count(']')
    if open_count > close_count:
        s += ']' * (open_count - close_count)
    return s

def _parse_triples(block: str):
    blk = _sanitize(block)
    blk = _balance_brackets(blk)    # ← auto-close any unbalanced lists
    for loader in (json.loads, ast.literal_eval):
        try:
            data = loader(blk)
            return [tuple(x) for x in data if len(x)==3]
        except:
            continue
    return None


def extract_triples(paragraph: str, generator) -> list[tuple[str,str,str]]:
    prompt = TRIPLE_PROMPT.format(paragraph=paragraph)
    out    = generator(prompt,
                      max_new_tokens=1200,
                      temperature=0.0,
                      do_sample=False)[0]["generated_text"]

    block = _first_json_list_after(out)
    if not block:
        print("No JSON block found after </TEXT>")
        return []

    triples = _parse_triples(block)
    if triples is None:
        print("Still could not parse block:\n", block[:200])
        return []
    return triples

# ------ 3. build the in-memory graph ---------------------------
def build_paragraph_kg(paragraph: str, generator) -> nx.MultiDiGraph:
    """Return a networkx MultiDiGraph representing the paragraph KG."""
    ents    = extract_entities(paragraph)
    triples = extract_triples(paragraph, generator)

    G = nx.MultiDiGraph()

    # add entity nodes
    for e in ents:
        G.add_node(e, type="entity")

    # add triples
    for h, r, t in triples:
        for node in (h, t):
            if node not in G:
                G.add_node(node, type="entity")
        G.add_edge(h, t, label=r)

    return G




# ------ 4. optional: push to Neo4j -----------------------------
CREATE_NODE = """
MERGE (c:ParaConcept {name:$name})
RETURN id(c) AS id
"""
CREATE_EDGE = """
MATCH (h:ParaConcept {name:$h}),
      (t:ParaConcept {name:$t})
MERGE (h)-[:PARA_REL {type:$rel}]->(t)
"""

def push_to_neo4j(G: nx.MultiDiGraph, driver: GraphDatabase.driver):
    with driver.session() as session:
        # nodes
        for n in G.nodes:
            session.run(CREATE_NODE, name=n)
        # edges
        for h, t, data in G.edges(data=True):
            session.run(CREATE_EDGE, h=h, t=t, rel=data.get("label", ""))


# -------------------------- demo -------------------------------
if __name__ == "__main__":
    from hybrid_search import generator, driver  # re-use your objects

    test_para = "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    kg = build_paragraph_kg(test_para, generator)
    print("Nodes:", kg.nodes(data=True))
    print("Edges:", list(kg.edges(data=True)))

    # Uncomment to persist in Neo4j
    # push_to_neo4j(kg, driver)


In [109]:
# This is hybrid search recommendation with users' historical data 
# But still, need to think about how to make it faster because now its running on cpu
# Need to think about importance score for the keywords matching - is solved 
# Need to think about path length. hops reasoning 

import os
import re
import json
import warnings
import pandas as pd
import torch
import numpy as np          


from neo4j import GraphDatabase
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore", category=UserWarning)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)

# This is the generation pipeline 
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Extract concepts with refined few-shot prompting
def extract_concepts(paragraph: str):
    prompt = f"""
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Respond only with a JSON object of the form {{"concepts": ["concept1","concept2",…]}}. Extract some unique terms rather than common words in computer science paper paragraph. 

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts": ["transformer", "BERT", "GPT", "bidirectional attention", "pretraining", "question answering", "machine translation", "text summarization"]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts": ["knowledge graph","entity linking", "question answering", "recommendation systems", "semantic context"]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).  
They’ve been applied to node classification, link prediction, and molecular property prediction.
</TEXT>
Expected output:
{{"concepts": ["graph neural network", "neighborhood aggregation", "Graph Convolutional Network (GCN)", "Graph Attention Network (GAT)", "Message Passing Neural Network (MPNN)", "node classification", "link prediction", "molecular property prediction"]}}

Example 4:
<TEXT>
To speed up query performance, modern database systems often employ B-tree and LSM-tree indexes.  
B-trees support balanced, ordered data access with logarithmic search time, while Log-Structured Merge trees buffer writes in memory and batch them to disk for high write throughput.  
Secondary indexes like inverted lists or hash indexes accelerate lookups on non-primary key columns.
</TEXT>
Expected output:
{{"concepts": ["B-tree index", "LSM-tree index", "logarithmic search time", "write buffering", "batch disk writes", "secondary index", "inverted list", "hash index", "non-primary key lookup"]}}

Example 5:
<TEXT>
In distributed consensus, Raft and Paxos are two foundational algorithms.  
Raft divides the problem into leader election, log replication, and safety, making it more understandable.  
Paxos focuses on proposer, acceptor, and learner roles to reach agreement despite failures.  
Gossip protocols and vector clock mechanisms are also widely used for state propagation and causality tracking.
</TEXT>
Expected output:
{{"concepts": ["distributed consensus", "Raft algorithm", "leader election", "log replication", "Paxos algorithm", "proposer role", "acceptor role", "learner role", "gossip protocol", "vector clock"]}}


---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON only, no extra text):

"""

    result = generator(
        prompt,
        max_new_tokens=600,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

# Max new tokens are set to 600 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want.


    # Extract valid JSON from output
    # This line try to search all strings like this (json-looking substrings) and (re is search for patterns)
    # The json file is made like this {"concepts": ["knowledge graphs", "AI systems", "data integration"]}
    matches = re.findall(r"\{[^{}]+\}", result, re.S)
    if matches:
        last = matches[-1]
        try:
            data = json.loads(last)
            if "concepts" in data and isinstance(data["concepts"], list):
                return data["concepts"]
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse model output for paragraph. Full output:\n{result}")


# ─────────────────────────────────────────────────────────────
# Group concepts by priority  (overlap | paragraph-only | prev-only)
def get_concept_sets(paragraph: str, prev_abstract: str):
    para_kw = set(extract_concepts(paragraph))
    prev_kw = set(extract_concepts(prev_abstract))

    overlap   = sorted(para_kw & prev_kw)          # weight 3
    para_only = sorted(para_kw - prev_kw)          # weight 2
    prev_only = sorted(prev_kw - para_kw)          # weight 1
    return overlap, para_only, prev_only
# ─────────────────────────────────────────────────────────────


# SciBERT embedding 
print("▶ Loading SciBERT embedder…")
sci_model = SentenceTransformer("allenai/scibert_scivocab_uncased")
sci_model.eval()

def embed(text: str) -> list[float]:
    # returns normalized embedding(Numpy array)
    # 768 floats are returned for vector index nearest neighbor search into neo4j. 
    vec = sci_model.encode(text, convert_to_numpy=True, normalize_embeddings=True)
    return vec.tolist()

# Instead of searching Neo4j HNSW index by paragraph alone, we blend the paragraph embedding with the author's abstract 
# Blend paragraph and previous-abstract embeddings
def blended_vec(paragraph: str, prev_abstract: str,
                w_para: float = 0.7, w_prev: float = 0.3) -> list[float]:
    v_para = np.array(embed(paragraph))
    v_prev = np.array(embed(prev_abstract))
    combo  = w_para * v_para + w_prev * v_prev
    combo /= np.linalg.norm(combo)       # re-normalize
    return combo.tolist()


def vector_search(qvec, top_k=25) -> pd.DataFrame:
    with driver.session() as s:
        rows = s.run(
            """
            CALL db.index.vector.queryNodes('paper_vec', $k, $vec)
            YIELD node, score
            RETURN node.id AS id, 1.0 - score AS sim
            ORDER BY score ASC
            """, k=top_k, vec=qvec
        ).data()
    return pd.DataFrame(rows)


def hydrate_paper_meta(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    ids = df["id"].tolist()
    with driver.session() as s:
        meta = s.run(
            "MATCH (p:Paper) WHERE p.id IN $ids RETURN p.id AS id, p.title AS title, p.year AS year",
            ids=ids
        ).data()
    return df.merge(pd.DataFrame(meta), on="id", how="left")[["id","title","year","sim"]]


# This is neo4j driver (calling neo4j)
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI",  "bolt://localhost:7687"),
    auth=(
        os.getenv("NEO4J_USER","neo4j"),
        os.getenv("NEO4J_PASS","Manami1008")
    )
)

# MATCH (p:Paper) selects all paper labels, and WHERE tells the paper which matches with concepts etraxted from user's paragraph.
# $terms is a parameter which is passed into
# So its like "Is there any term t in the input $terms list such that lowercase paper title contains that term?"
# The keywords are seached in title, abstracts, and field of study or topics. 
# But for as topics and as field of study (x), it checks one hop to search paper p
# Return paper id, title, year (now, its sorted by year cuz there is no ranking method)

BASE_CYPHER = """
MATCH (p:Paper)
WHERE 
  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR
  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR
  EXISTS {
    MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)
    WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)
  }
RETURN p.id AS id, p.title AS title, p.year AS year
ORDER BY p.year DESC
"""

# The concept in this is accept a list of concept keywords extracted from a paragraph
# And run Cypher query agianst Neo4j knowledge graph
# Return a paper
# It returns a pandas.DataFrame object containing search results from Neo4j.
# Filters out single word tems by keeping 2 o more words to reduce noise. Multi-word is always better no?
# After filltering it out returns dataFrame
# rows = s.run(BASE_CYPHER, terms=terms).data() this executes the BASE_CYPHER query and the pass the terms list into $terms


def graph_search(concepts: list[str]) -> pd.DataFrame:
    terms = [c.lower() for c in concepts if len(c.split()) >= 2]
    if not terms:
        return pd.DataFrame(columns=["id","title","year"])
    with driver.session() as s:
        rows = s.run(BASE_CYPHER, terms=terms).data()
    return pd.DataFrame(rows)

# Weighted merge of three priority groups
def weighted_graph_search(overlap, para_only, prev_only,
                           top_k_each=100, w=(3,2,1)) -> pd.DataFrame:

    def _query(terms):
        if not terms:         # empty list → empty DataFrame
            return pd.DataFrame(columns=["id","title","year"])
        return graph_search(terms).head(top_k_each)

    df_o = _query(overlap);    df_o["w"] = w[0]
    df_p = _query(para_only);  df_p["w"] = w[1]
    df_r = _query(prev_only);  df_r["w"] = w[2]

    big = pd.concat([df_o, df_p, df_r], ignore_index=True)
    if big.empty:
        return big             # nothing matched

    big["hit"] = 1
    scored = (big.groupby(["id","title","year"], as_index=False)
                   .agg(score=("w","sum"), hits=("hit","count"))
                   .sort_values(["score","year"], ascending=[False,False]))
    return scored.head(200)

# Demo
if __name__ == "__main__":
    paragraph = (
       "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    )

    prev_abstract = (
    "With the prosperity of e-commerce and web applications, Recommender Systems (RecSys) have become an important component of our daily life, providing personalized suggestions that cater to user preferences. While Deep Neural Networks (DNNs) have made significant advancements in enhancing recommender systems by modeling user-item interactions and incorporating textual side information, DNN-based methods still face limitations, such as difficulties in understanding users' interests and capturing textual side information, inabilities in generalizing to various recommendation scenarios and reasoning on their predictions, etc. Meanwhile, the emergence of Large Language Models (LLMs), such as ChatGPT and GPT4, has revolutionized the fields of Natural Language Processing (NLP) and Artificial Intelligence (AI), due to their remarkable abilities in fundamental responsibilities of language understanding and generation, as well as impressive generalization and reasoning capabilities. As a result, recent studies have attempted to harness the power of LLMs to enhance recommender systems. Given the rapid evolution of this research direction in recommender systems, there is a pressing need for a systematic overview that summarizes existing LLM-empowered recommender systems, to provide researchers in relevant fields with an in-depth understanding. Therefore, in this paper, we conduct a comprehensive review of LLM-empowered recommender systems from various aspects including Pre-training, Fine-tuning, and Prompting. More specifically, we first introduce representative methods to harness the power of LLMs (as a feature encoder) for learning representations of users and items. Then, we review recent techniques of LLMs for enhancing recommender systems from three paradigms, namely pre-training, fine-tuning, and prompting. Finally, we comprehensively discuss future directions in this emerging field. "
    )
    
    # -- extract concept groups
    overlap, para_only, prev_only = get_concept_sets(paragraph, prev_abstract)
    
    # -- keyword-based graph search with weights
    df_kw = weighted_graph_search(overlap, para_only, prev_only)
    print("\n Top keyword hits:")
    print(df_kw.head(20).to_string(index=False))
    
    # -- blended vector search
    vcombo = blended_vec(paragraph, prev_abstract)
    df_vec  = vector_search(vcombo, top_k=50)
    df_vec  = hydrate_paper_meta(df_vec)
    print("\n Top vector hits:")
    print(df_vec.head(20).to_string(index=False, formatters={"sim":"{:.3f}".format}))



▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.66it/s]
Device set to use cpu
No sentence-transformers model found with name allenai/scibert_scivocab_uncased. Creating a new one with mean pooling.


▶ Loading SciBERT embedder…


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_TOPIC|HAS_FOS' instead)} {position: line: 7, column: 26, offset: 178} for query: '\nMATCH (p:Paper)\nWHERE \n  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR\n  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR\n  EXISTS {\n    MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)\n    WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)\n  }\nRETURN p.id AS id, p.title AS title, p.year AS year\nORDER BY p.year DESC\n'
Received notification from DBMS server: {severity: WARN


⚡ Top keyword hits:
        id                                                                                                                              title  year  score  hits
2914397182                                   Evaluating the state-of-the-art of End-to-End Natural Language Generation: The E2E NLG challenge  2020      4     2
2599685942                                                          Gaze Interaction With Vibrotactile Feedback: Review and Design Guidelines  2020      3     2
2805294773                                                                     A Unified Latent Variable Model for Contrastive Opinion Mining  2020      3     2
2886493354                                                          A syntactic path-based hybrid neural network for negation scope detection  2020      3     2
2888338418                                                                                   Noiseprint: A CNN-Based Camera Model Fingerprint  2020      3     2
2888382142   

With the prosperity of e-commerce and web applications, Recommender Systems (RecSys) have become an important component of our daily life, providing personalized suggestions that cater to user preferences. While Deep Neural Networks (DNNs) have made significant advancements in enhancing recommender systems by modeling user-item interactions and incorporating textual side information, DNN-based methods still face limitations, such as difficulties in understanding users' interests and capturing textual side information, inabilities in generalizing to various recommendation scenarios and reasoning on their predictions, etc. Meanwhile, the emergence of Large Language Models (LLMs), such as ChatGPT and GPT4, has revolutionized the fields of Natural Language Processing (NLP) and Artificial Intelligence (AI), due to their remarkable abilities in fundamental responsibilities of language understanding and generation, as well as impressive generalization and reasoning capabilities. As a result, recent studies have attempted to harness the power of LLMs to enhance recommender systems. Given the rapid evolution of this research direction in recommender systems, there is a pressing need for a systematic overview that summarizes existing LLM-empowered recommender systems, to provide researchers in relevant fields with an in-depth understanding. Therefore, in this paper, we conduct a comprehensive review of LLM-empowered recommender systems from various aspects including Pre-training, Fine-tuning, and Prompting. More specifically, we first introduce representative methods to harness the power of LLMs (as a feature encoder) for learning representations of users and items. Then, we review recent techniques of LLMs for enhancing recommender systems from three paradigms, namely pre-training, fine-tuning, and prompting. Finally, we comprehensively discuss future directions in this emerging field. 

In [110]:
# This is hybrid search recommendation with users' historical data 
# But still, need to think about how to make it faster because now its running on cpu
# Need to think about importance score for the keywords matching - is solved 
# Need to think about path length. hops reasoning 

import os
import re
import json
import warnings
import pandas as pd
import torch

from neo4j import GraphDatabase
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore", category=UserWarning)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)

# This is the generation pipeline 
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Extract concepts with refined few-shot prompting
def extract_concepts(paragraph: str):
    prompt = f"""
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Respond only with a JSON object of the form {{"concepts": ["concept1","concept2",…]}}. Extract some unique terms rather than common words in computer science paper paragraph. 

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts": ["transformer", "BERT", "GPT", "bidirectional attention", "pretraining", "question answering", "machine translation", "text summarization"]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts": ["knowledge graph","entity linking", "question answering", "recommendation systems", "semantic context"]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).  
They’ve been applied to node classification, link prediction, and molecular property prediction.
</TEXT>
Expected output:
{{"concepts": ["graph neural network", "neighborhood aggregation", "Graph Convolutional Network (GCN)", "Graph Attention Network (GAT)", "Message Passing Neural Network (MPNN)", "node classification", "link prediction", "molecular property prediction"]}}

Example 4:
<TEXT>
To speed up query performance, modern database systems often employ B-tree and LSM-tree indexes.  
B-trees support balanced, ordered data access with logarithmic search time, while Log-Structured Merge trees buffer writes in memory and batch them to disk for high write throughput.  
Secondary indexes like inverted lists or hash indexes accelerate lookups on non-primary key columns.
</TEXT>
Expected output:
{{"concepts": ["B-tree index", "LSM-tree index", "logarithmic search time", "write buffering", "batch disk writes", "secondary index", "inverted list", "hash index", "non-primary key lookup"]}}

Example 5:
<TEXT>
In distributed consensus, Raft and Paxos are two foundational algorithms.  
Raft divides the problem into leader election, log replication, and safety, making it more understandable.  
Paxos focuses on proposer, acceptor, and learner roles to reach agreement despite failures.  
Gossip protocols and vector clock mechanisms are also widely used for state propagation and causality tracking.
</TEXT>
Expected output:
{{"concepts": ["distributed consensus", "Raft algorithm", "leader election", "log replication", "Paxos algorithm", "proposer role", "acceptor role", "learner role", "gossip protocol", "vector clock"]}}


---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON only, no extra text):

"""

    result = generator(
        prompt,
        max_new_tokens=600,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

# Max new tokens are set to 600 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want.


    # Extract valid JSON from output
    # This line try to search all strings like this (json-looking substrings) and (re is search for patterns)
    # The json file is made like this {"concepts": ["knowledge graphs", "AI systems", "data integration"]}
    matches = re.findall(r"\{[^{}]+\}", result, re.S)
    if matches:
        last = matches[-1]
        try:
            data = json.loads(last)
            if "concepts" in data and isinstance(data["concepts"], list):
                return data["concepts"]
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse model output for paragraph. Full output:\n{result}")

# Call the extract_concepts LLaMA prompt 
def get_concepts(text: str) -> set[str]:
    return set(map(str.lower, extract_concepts(text)))

def compare_concept_sets(cur_paragraph: str, prev_abstract: str):
    cur_set  = get_concepts(cur_paragraph)
    prev_set = get_concepts(prev_abstract)

    common      = sorted(cur_set & prev_set)          # P₁
    new_only    = sorted(cur_set - prev_set)          # P₂
    legacy_only = sorted(prev_set - cur_set)          # P₃

    return common, new_only, legacy_only


# SciBERT embedding 
print("▶ Loading SciBERT embedder…")
sci_model = SentenceTransformer("allenai/scibert_scivocab_uncased")
sci_model.eval()

def embed(text: str) -> list[float]:
    # returns normalized embedding
    vec = sci_model.encode(text, convert_to_numpy=True, normalize_embeddings=True)
    return vec.tolist()

def vector_search(qvec, top_k=25) -> pd.DataFrame:
    with driver.session() as s:
        rows = s.run(
            """
            CALL db.index.vector.queryNodes('paper_vec', $k, $vec)
            YIELD node, score
            RETURN node.id AS id, 1.0 - score AS sim
            ORDER BY score ASC
            """, k=top_k, vec=qvec
        ).data()
    return pd.DataFrame(rows)


def hydrate_paper_meta(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    ids = df["id"].tolist()
    with driver.session() as s:
        meta = s.run(
            "MATCH (p:Paper) WHERE p.id IN $ids RETURN p.id AS id, p.title AS title, p.year AS year",
            ids=ids
        ).data()
    return df.merge(pd.DataFrame(meta), on="id", how="left")[["id","title","year","sim"]]


# This is neo4j driver (calling neo4j)
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI",  "bolt://localhost:7687"),
    auth=(
        os.getenv("NEO4J_USER","neo4j"),
        os.getenv("NEO4J_PASS","Manami1008")
    )
)

# MATCH (p:Paper) selects all paper labels, and WHERE tells the paper which matches with concepts etraxted from user's paragraph.
# $terms is a parameter which is passed into
# So its like "Is there any term t in the input $terms list such that lowercase paper title contains that term?"
# The keywords are seached in title, abstracts, and field of study or topics. 
# But for as topics and as field of study (x), it checks one hop to search paper p
# Return paper id, title, year (now, its sorted by year cuz there is no ranking method)

BASE_CYPHER = """
MATCH (p:Paper)
WHERE 
  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR
  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR
  EXISTS {
    MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)
    WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)
  }
RETURN p.id AS id, p.title AS title, p.year AS year
ORDER BY p.year DESC
"""

WEIGHTED_CYPHER = """
UNWIND $terms AS item
WITH item.term  AS term,
     item.w     AS w
MATCH (p:Paper)
WHERE toLower(p.title)    CONTAINS term OR
      toLower(p.abstract) CONTAINS term OR
      EXISTS {
        MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)
        WHERE toLower(x.name) CONTAINS term
      }
WITH p, max(w) AS wt               // if multiple terms hit, keep the highest weight
RETURN p.id   AS id,
       p.title AS title,
       p.year  AS year,
       sum(wt) AS importance       // aggregate across different terms
ORDER BY importance DESC, year DESC
LIMIT 200
"""

def weighted_graph_search(common, new_only, legacy_only, w_common=1.0, w_new=0.7, w_old=0.4):
    # Build [{term: "...", w: 1.0}, …] but keep only multi-word to reduce noise
    pack = (
        [{"term": t, "w": w_common} for t in common     if len(t.split()) >= 2] +
        [{"term": t, "w": w_new   } for t in new_only   if len(t.split()) >= 2] +
        [{"term": t, "w": w_old   } for t in legacy_only if len(t.split()) >= 2]
    )
    if not pack:
        return pd.DataFrame(columns=["id","title","year","importance"])
    with driver.session() as s:
        rows = s.run(WEIGHTED_CYPHER, terms=pack).data()
    return pd.DataFrame(rows)


# The concept in this is accept a list of concept keywords extracted from a paragraph
# And run Cypher query agianst Neo4j knowledge graph
# Return a paper
# It returns a pandas.DataFrame object containing search results from Neo4j.
# Filters out single word tems by keeping 2 o more words to reduce noise. Multi-word is always better no?
# After filltering it out returns dataFrame
# rows = s.run(BASE_CYPHER, terms=terms).data() this executes the BASE_CYPHER query and the pass the terms list into $terms


def graph_search(concepts: list[str]) -> pd.DataFrame:
    terms = [c.lower() for c in concepts if len(c.split()) >= 2]
    if not terms:
        return pd.DataFrame(columns=["id","title","year"])
    with driver.session() as s:
        rows = s.run(BASE_CYPHER, terms=terms).data()
    return pd.DataFrame(rows)


if __name__ == "__main__":
    paragraph = """ Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems. """
    prev_abs  = """ With the prosperity of e-commerce and web applications, Recommender Systems (RecSys) have become an important component of our daily life, providing personalized suggestions that cater to user preferences. While Deep Neural Networks (DNNs) have made significant advancements in enhancing recommender systems by modeling user-item interactions and incorporating textual side information, DNN-based methods still face limitations, such as difficulties in understanding users' interests and capturing textual side information, inabilities in generalizing to various recommendation scenarios and reasoning on their predictions, etc. Meanwhile, the emergence of Large Language Models (LLMs), such as ChatGPT and GPT4, has revolutionized the fields of Natural Language Processing (NLP) and Artificial Intelligence (AI), due to their remarkable abilities in fundamental responsibilities of language understanding and generation, as well as impressive generalization and reasoning capabilities. As a result, recent studies have attempted to harness the power of LLMs to enhance recommender systems. Given the rapid evolution of this research direction in recommender systems, there is a pressing need for a systematic overview that summarizes existing LLM-empowered recommender systems, to provide researchers in relevant fields with an in-depth understanding. Therefore, in this paper, we conduct a comprehensive review of LLM-empowered recommender systems from various aspects including Pre-training, Fine-tuning, and Prompting. More specifically, we first introduce representative methods to harness the power of LLMs (as a feature encoder) for learning representations of users and items. Then, we review recent techniques of LLMs for enhancing recommender systems from three paradigms, namely pre-training, fine-tuning, and prompting. Finally, we comprehensively discuss future directions in this emerging field. """

    # 1 Extract & compare
    p1, p2, p3 = compare_concept_sets(paragraph, prev_abs)

    print("\n■ Concepts extracted")
    print("P₁ (common) :", p1)
    print("P₂ (new)    :", p2)
    print("P₃ (old)    :", p3)

    # 2 Run weighted graph search
    df_w = weighted_graph_search(p1, p2, p3)
    print(f"\n⬇ Weighted graph search ({len(df_w)} hits):")
    print(df_w.head(25).to_string(index=False))

    # 3 Vector search on current paragraph (unchanged)
    df_vec = vector_search(embed(paragraph), top_k=25)
    df_vec = hydrate_paper_meta(df_vec)
    print(f"\n⬇ Vector search ({len(df_vec)} hits):")
    print(df_vec.head(25).to_string(index=False, formatters={"sim":"{:.3f}".format}))



▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.88it/s]
Device set to use cpu
No sentence-transformers model found with name allenai/scibert_scivocab_uncased. Creating a new one with mean pooling.


▶ Loading SciBERT embedder…


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



■ Concepts extracted
P₁ (common) : ['generalization', 'language generation', 'language understanding', 'large language models', 'natural language processing', 'pre-training', 'recommender systems']
P₂ (new)    : ['chain-of-thought', 'computer vision', 'human-like responses', 'in-context learning', 'molecule discovery', 'parameter size', 'prompting strategies', 'reasoning capabilities', 'training corpus', 'transformer-based models']
P₃ (old)    : ['artificial intelligence', 'chatgpt', 'deep neural networks', 'feature encoder', 'fine-tuning', 'gpt4', 'item representation', 'llm-empowered recommender systems', 'prompting', 'reasoning', 'representation learning', 'textual side information', 'user representation', 'user-item interactions']


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_TOPIC|HAS_FOS' instead)} {position: line: 9, column: 30, offset: 215} for query: '\nUNWIND $terms AS item\nWITH item.term  AS term,\n     item.w     AS w\nMATCH (p:Paper)\nWHERE toLower(p.title)    CONTAINS term OR\n      toLower(p.abstract) CONTAINS term OR\n      EXISTS {\n        MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)\n        WHERE toLower(x.name) CONTAINS term\n      }\nWITH p, max(w) AS wt               // if multiple terms hit, keep the highest weight\nRETURN p.id   AS id,\n       p.title AS title,\n       p.year  AS year,\n       sum(wt) AS importance       // aggregate across different terms\nORDER BY importance 


⬇ Weighted graph search (200 hits):
        id                                                                                                                                                title  year  importance
2998040827                                                                                                                             Voice assistance in 2019  2020         1.0
3004133200                                           Comparing Rule-based, Feature-based and Deep Neural Methods for De-identification of Dutch Medical Records  2020         1.0
2990024321                                                          What is the minimum training data size to reliably identify writers in medieval manuscripts  2020         1.0
2998037504                                                                                            Entropy of Polysemantic Words for the Same Part of Speech  2020         1.0
2999927792                                                         A Hybr

In [120]:

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, re, json

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading Llama-3 8B…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
                MODEL_ID,
                torch_dtype=torch.float16,  # I think this is memmory efficient, less VRAM usage!!
                device_map="auto"  #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
            )

# This is the generation pipeline 
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Extract concepts with chain of thought few-shot prompting. 
# https://www.promptingguide.ai/techniques/cot
# https://www.mercity.ai/blog-post/guide-to-chain-of-thought-prompting#what-is-chain-of-thought-prompting
def extract_context(paragraph: str) -> dict:
    prompt = f"""
You are an academic assistant an elite writing assistant for computer-science research.
You need to understand the context of paragraph given very well like human. 


TASK
Step 1 – THINK
  • Read the paragraph in <TEXT>.
  • Deliberate step-by-step inside the tags <COT> … </COT>.

Step 2 – EXTRACT six fields
  1) main_topic        – up to 2 short keywords (list)
  2) subtopics         – up to 5 phrases (list)
  3) problem_statement – ONE sentence (≤ 30 tokens)
  4) technologies      – list concrete models / algorithms / datasets
  5) research_domain   – broad area (e.g. “machine learning”)
  6) user_intent       – 5-15 tokens (“survey X”, “find gaps in Y”, …)

Step 3 – OUTPUT
  • Write valid compact JSON only.
  • Keys must appear in the order shown above.

RULES
  • Read carefully with understanding of paragraph‐level cohesion and local coherence.
  • Do NOT invent papers, references, or details not implied by the text.
  • Outside <COT> and the final JSON, write nothing else.

EXAMPLES

Example 1
<COT>
• Identify domain terms: “BERT”, “GPT”, “bidirectional attention”.
• Core area ≈ “Transformer-based NLP” → main_topic.
• Sub-areas: pretraining, question answering, machine translation.
• Problem: traditional seq-models lack deep context.
• Tech list = {{BERT, GPT}}.
• Domain = NLP.
• Intent ≈ “survey modern transformer NLP work”.
</COT>
{{"main_topic":"transformer-based NLP",
  "subtopics":["pretraining","question answering","machine translation"],
  "problem_statement":"Sequence models before transformers struggled to capture long-range context in language tasks.",
  "technologies":["BERT","GPT"],
  "research_domain":"natural language processing",
  "user_intent":"survey modern transformer literature"}}

Example 2
<COT>
• Key concepts: “knowledge graph”, “entity linking”, “question answering”.
• main_topic = knowledge graphs.
• Subtopics: entity linking, QA, recommendation.
• Tech list none explicit → empty list.
• Problem stmt: structuring entities for downstream tasks.
• Domain: AI / NLP.
• Intent: discover KG applications.
</COT>
{{"main_topic":"knowledge graphs",
  "subtopics":["entity linking","question answering","recommendation systems"],
  "problem_statement":"Researchers need structured representations of entities and relations to enhance downstream tasks.",
  "technologies":[],
  "research_domain":"AI / information extraction",
  "user_intent":"discover KG application papers"}}

Example 3
<COT>
• Mentions: “graph neural networks”, “GCN”, “GAT”, “MPNN”.
• main_topic = graph neural networks.
• Subtopics: node classification, link prediction, molecular property.
• Tech list = {{GCN, GAT, MPNN}}.
• Problem stmt derived.
• Domain: machine learning.
• Intent: compare GNN variants.
</COT>
{{"main_topic":"graph neural networks",
  "subtopics":["node classification","link prediction","molecular property prediction"],
  "problem_statement":"Existing deep-learning methods need adaptation to non-Euclidean graph data structures.",
  "technologies":["Graph Convolutional Network","Graph Attention Network","Message Passing Neural Network"],
  "research_domain":"machine learning",
  "user_intent":"compare GNN variants"}}

 Example 4
<COT>
• Read full paragraph: it describes why context‐aware recommendation is important for academic writing.
• Sentence 1 mentions “context‐aware recommendations” → main_topic.
• Sentence 2 contrasts “traditional systems” based on citation networks vs. a need for paragraph‐level signals → problem.
• Sentence 3 says “we propose combining SciBERT embeddings of input text with a Neo4j knowledge graph” → technology list includes SciBERT, Neo4j.
• Sentence 4 adds “graph neural networks” and “retrieval‐augmented reranking” → subtopics include graph neural networks, retrieval‐augmented reranking.
• Domain is NLP / recommender systems.
• Intent: find papers on context‐aware KG retrieval.
</COT>
{{"main_topic":["context-aware recommendations"],
  "subtopics":["graph neural networks","retrieval-augmented reranking"],
  "problem_statement":"Traditional recommendation systems based on citation networks fail to capture paragraph-level shifts during academic writing.",
  "technologies":["SciBERT","Neo4j"],
  "research_domain":"natural language processing / recommender systems",
  "user_intent":"find papers on paragraph-level context-aware recommendation"}}

Example 5
<COT>
• Entire paragraph discusses constructing a knowledge graph from a single paragraph.
• It says “extract entities via spaCy NER”, “resolve coreference”, “extract relational triples via LLaMA prompting” → these are technologies.
• main_topic = paragraph knowledge-graph construction.
• Subtopics: entity extraction, coreference resolution, triple extraction.
• Problem: Building a mini‐KG on CPU is slow and path‐length reasoning is a challenge.
• Domain: knowledge graphs / NLP.
• Intent: discover KG optimization papers.
</COT>
{{"main_topic":["paragraph knowledge-graph construction"],
  "subtopics":["entity extraction","coreference resolution","relational triple extraction"],
  "problem_statement":"Constructing a mini‐knowledge graph from a paragraph on CPU is slow and inefficient for multi‐hop reasoning.",
  "technologies":["spaCy NER","neuralcoref","LLaMA prompting"],
  "research_domain":"knowledge graphs / natural language processing",
  "user_intent":"find papers on efficient paragraph-level KG construction"}}

Example 6
<TEXT>
Recent advances in healthcare AI have underscored the need for privacy-preserving machine-learning pipelines that comply with strict data-sharing regulations. Federated learning lets hospitals collaboratively train diagnostic models by exchanging only gradient updates, yet vanilla schemes remain vulnerable to membership-inference attacks. To mitigate this, researchers combine differential-privacy noise and secure-aggregation protocols while exploring homomorphic encryption for gradient masking. Unfortunately, these defenses often degrade model accuracy or impose heavy computational costs, hindering deployment across resource-heterogeneous clinics. This work introduces an adaptive federated-optimization algorithm that dynamically tunes privacy budgets and compression rates according to each client’s hardware profile, maintaining GDPR-level guarantees without sacrificing performance on chest-X-ray classification. Experiments across three global healthcare networks show a 12 % accuracy gain over fixed-budget baselines while meeting privacy constraints.
</TEXT>

<COT>
• Central theme: “privacy-preserving federated learning in healthcare” → main_topic.  
• Subtopics: differential privacy, secure aggregation, homomorphic encryption, adaptive optimization, healthcare deployment.  
• Problem: Existing privacy defenses hurt accuracy or add heavy compute overhead.  
• Technologies explicitly named: federated learning, differential privacy, secure aggregation, homomorphic encryption, adaptive federated optimization.  
• Research domain: machine learning / healthcare AI.  
• User intent: find papers on practical, privacy-preserving FL solutions.
</COT>
{{"main_topic":["federated learning"],
  "subtopics":["differential privacy","secure aggregation","homomorphic encryption","adaptive optimization","healthcare deployment"],
  "problem_statement":"Current privacy defenses in federated learning reduce accuracy or cause heavy computation in hospital settings.",
  "technologies":["federated learning","differential privacy","secure aggregation","homomorphic encryption","adaptive federated optimization"],
  "research_domain":"machine learning / healthcare AI",
  "user_intent":"find papers on privacy-preserving federated learning"}}


PARAGRAPH
<TEXT>
{paragraph}
</TEXT>

# After thinking in <COT>, output your JSON on a new line.
"""

# Max new tokens are set to 600 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want.
    raw = generator(
        prompt,
        max_new_tokens=1000,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

    # 2) Locate the *last* `{` and the *last* `}` in raw
    last_open = raw.rfind("{")
    last_close = raw.rfind("}")
    if last_open == -1 or last_close == -1 or last_close < last_open:
        raise ValueError("Extractor failed – could not find a well-formed JSON block.\n\nRaw output:\n" + raw)

    last_json_str = raw[last_open : last_close + 1]

    # 3) Parse it
    try:
        data = json.loads(last_json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON returned:\n{last_json_str}\n\nRaw output:\n{raw}") from e

    # 4) Sanity check required keys
    required = {
        "main_topic",
        "subtopics",
        "problem_statement",
        "technologies",
        "research_domain",
        "user_intent"
    }
    missing = required - set(data.keys())
    if missing:
        raise ValueError(f"Missing keys in returned JSON: {missing}\n\nReturned JSON:\n{last_json_str}")

    return data


# Demo
if __name__ == "__main__":
    paragraph = (
        "Recently, as advanced natural language processing techniques, "
        "Large Language Models (LLMs) with billion parameters have generated "
        "large impacts on various research fields such as NLP, Computer Vision, "
        "and Molecule Discovery. Technically most existing LLMs are transformer-"
        "based models pre-trained on a vast amount of textual data from diverse sources. "
        "As the parameter size of LLMs continues to scale, recent studies indicated that LLMs "
        "can lead to the emergence of remarkable capabilities. More specifically, LLMs demonstrate "
        "powerful abilities in language understanding and generation, enabling them to better "
        "comprehend human intentions. Moreover, LLMs exhibit impressive generalization and "
        "reasoning, often applying learned knowledge to new tasks with few-shot demonstrations."
    )

    ctx = extract_context(paragraph)
    print("Extracted context:")
    for k, v in ctx.items():
        print(f"{k:18}: {v}")



▶ Loading Llama-3 8B…


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.96it/s]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Extracted context:
main_topic        : ['Large Language Models']
subtopics         : ['transformer-based models', 'pre-training', 'language understanding', 'language generation', 'generalization', 'reasoning']
problem_statement : Existing LLMs struggle to capture human intentions and generalize to new tasks.
technologies      : ['transformer-based models']
research_domain   : natural language processing
user_intent       : survey recent LLM advancements
